In [ ]:
#This is an example of using the Medical Expenditure Panel Survey (MEPS), a longitudinal data set from
#2019-22 in Python. The file was too big to automatically import in SQL, so I began the project in Python to reduce
#the file size and keep only the required columns for data analysis.

import pandas as pd
import numpy as np

# Define file paths
input_file = "/content/meps19_22.csv"  # Update if needed
output_file = "/content/meps19_22_clean.csv"

# Define columns to keep
columns_to_keep = [
    "DUPERSID", "AGEY1X", "AGEY2X", "AGEY3X", "AGEY4X",
    "FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4",
    "TOTEXPY1", "TOTEXPY2", "TOTEXPY3", "TOTEXPY4",
    "TOTSLFY1", "TOTSLFY2", "TOTSLFY3", "TOTSLFY4",
    "OBTOTVY1", "OBTOTVY2", "OBTOTVY3", "OBTOTVY4",
    "ERTOTY1", "ERTOTY2", "ERTOTY3", "ERTOTY4",
    "IPDISY1", "IPDISY2", "IPDISY3", "IPDISY4",
    "RXEXPY1", "RXEXPY2", "RXEXPY3", "RXEXPY4",
    "INSURCY1", "INSURCY2", "INSURCY3", "INSURCY4",
    "LONGWT"
]

# Load dataset
df = pd.read_csv(input_file, usecols=columns_to_keep)

# ✅ Recode Insurance Groups (Public, Private, Uninsured)
def recode_insurance(value):
    if value in [1]:
        return "Private"
    elif value in [2, 4, 5, 6]:
        return "Public"
    elif value in [3, 7, 8]:
        return "Uninsured"
    else:
        return None

for year in range(1, 5):
    df[f"INSUR_GROUPY{year}"] = df[f"INSURCY{year}"].apply(recode_insurance)

# Recode Income into Groups
income_bins = [0, 25000, 50000, 75000, 100000, float("inf")]
income_labels = ["Low (<25K)", "Lower-Middle (25K-50K)", "Middle (50K-75K)", "Upper-Middle (75K-100K)", "High (>100K)"]

for year in range(1, 5):
    df[f"INCOME_GROUPY{year}"] = pd.cut(df[f"FAMINCY{year}"], bins=income_bins, labels=income_labels)

# Recode Age into Groups
age_bins = [0, 18, 35, 50, 65, float("inf")]
age_labels = ["0-17", "18-34", "35-49", "50-64", "65+"]

for year in range(1, 5):
    df[f"AGE_GROUPY{year}"] = pd.cut(df[f"AGEY{year}X"], bins=age_bins, labels=age_labels)

# Replace -1 values with NaN for missing data
df.replace(-1, np.nan, inplace=True)

# Save cleaned dataset
df.to_csv(output_file, index=False)
print(f"✅ Cleaned dataset saved as: {output_file}")

# Check for missing values
print("\nMissing Values Summary:\n", df.isna().sum())



#Note: Most variables have missing values (-1) and the data is in wide format.
#We'll address this in the next section.



✅ Cleaned dataset saved as: /content/meps19_22_clean.csv

Missing Values Summary:
 DUPERSID            0
AGEY1X             68
AGEY2X            104
AGEY3X            124
AGEY4X            133
FAMINCY1           26
FAMINCY2           73
FAMINCY3          107
FAMINCY4          121
INSURCY1           26
INSURCY2           73
INSURCY3          107
INSURCY4          121
TOTEXPY1           26
TOTEXPY2           73
TOTEXPY3          107
TOTEXPY4          121
TOTSLFY1           26
TOTSLFY2           73
TOTSLFY3          107
TOTSLFY4          121
OBTOTVY1           26
OBTOTVY2           73
OBTOTVY3          107
OBTOTVY4          121
ERTOTY1            26
ERTOTY2            73
ERTOTY3           107
ERTOTY4           121
IPDISY1            26
IPDISY2            73
IPDISY3           107
IPDISY4           121
RXEXPY1            26
RXEXPY2            73
RXEXPY3           107
RXEXPY4           121
LONGWT              1
INSUR_GROUPY1      26
INSUR_GROUPY2      73
INSUR_GROUPY3     107
INSUR_GROUPY4  

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "/content/meps19_22_clean.csv"
df = pd.read_csv(file_path)

# Identify categorical and continuous variables
categorical_cols = ["INSURCY1", "INSURCY2", "INSURCY3", "INSURCY4"]
continuous_cols = [
    "AGEY1X", "AGEY2X", "AGEY3X", "AGEY4X",
    "FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4",
    "TOTEXPY1", "TOTEXPY2", "TOTEXPY3", "TOTEXPY4",
    "TOTSLFY1", "TOTSLFY2", "TOTSLFY3", "TOTSLFY4",
    "OBTOTVY1", "OBTOTVY2", "OBTOTVY3", "OBTOTVY4",
    "ERTOTY1", "ERTOTY2", "ERTOTY3", "ERTOTY4",
    "IPDISY1", "IPDISY2", "IPDISY3", "IPDISY4",
    "RXEXPY1", "RXEXPY2", "RXEXPY3", "RXEXPY4",
    "LONGWT"
]

# Check for missing (-1) values
missing_values = (df == -1).sum()
print("\nColumns with missing values:\n", missing_values[missing_values > 0])

# Count rows with at least one missing (-1) value
rows_with_neg1 = (df == -1).any(axis=1).sum()
total_rows = len(df)
percent_removed = (rows_with_neg1 / total_rows) * 100

print(f"\nTotal rows in dataset: {total_rows}")
print(f"Rows with at least one -1: {rows_with_neg1}")
print(f"Percentage of dataset that would be removed: {percent_removed:.2f}%")

# Check missingness for categorical and continuous variables separately
missing_categorical = df[categorical_cols].isin([-1]).any(axis=1).sum()
missing_continuous = df[continuous_cols].isin([-1]).any(axis=1).sum()

print(f"\nCases with missing categorical variables: {missing_categorical}")
print(f"Cases with missing continuous variables: {missing_continuous}")

# **Final Decision Printout**
print(f"\nIf we remove rows with missing categorical data, we will lose {missing_categorical} cases.")
print(f"If we impute continuous variables, we will retain {total_rows - missing_categorical} cases.")

# Next Step: Proceed with Hybrid Approach (Dropping categorical missing cases & imputing continuous ones)


#If we removing cases with missing data, we would lose about 9% of the data.
#We could go with complete case analysis here but we will do a hybrid approach
#to impute continiuous variables and remove categorical variables with missing data.



Columns with missing values:
 Series([], dtype: int64)

Total rows in dataset: 1837
Rows with at least one -1: 0
Percentage of dataset that would be removed: 0.00%

Cases with missing categorical variables: 0
Cases with missing continuous variables: 0

If we remove rows with missing categorical data, we will lose 0 cases.
If we impute continuous variables, we will retain 1837 cases.


In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer  # Required for Google Colab
from sklearn.impute import IterativeImputer

# Replace -1 values with NaN for imputation
df.replace(-1, np.nan, inplace=True)

# Identify categorical and continuous variables
categorical_cols = ["INSURCY1", "INSURCY2", "INSURCY3", "INSURCY4"]
continuous_cols = [
    "AGEY1X", "AGEY2X", "AGEY3X", "AGEY4X",
    "FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4",
    "TOTEXPY1", "TOTEXPY2", "TOTEXPY3", "TOTEXPY4",
    "TOTSLFY1", "TOTSLFY2", "TOTSLFY3", "TOTSLFY4",
    "OBTOTVY1", "OBTOTVY2", "OBTOTVY3", "OBTOTVY4",
    "ERTOTY1", "ERTOTY2", "ERTOTY3", "ERTOTY4",
    "IPDISY1", "IPDISY2", "IPDISY3", "IPDISY4",
    "RXEXPY1", "RXEXPY2", "RXEXPY3", "RXEXPY4",
    "LONGWT"
]

# **Step 1: Drop Rows with Missing Categorical Variables**
df = df.dropna(subset=categorical_cols)

# **Step 2: Impute Continuous Variables Using Regression-Based Imputation**
def regression_impute(df, target, predictors):
    """Impute missing values using Linear Regression."""
    df_nonmissing = df.dropna(subset=[target] + predictors)
    df_missing = df[df[target].isna()]

    if df_missing.empty:
        return df[target]  # No missing values to impute

    model = LinearRegression()
    model.fit(df_nonmissing[predictors], df_nonmissing[target])
    df.loc[df[target].isna(), target] = model.predict(df_missing[predictors])
    return df[target]

# Apply regression imputation for key continuous variables
for col in continuous_cols:
    predictors = [c for c in continuous_cols if c != col and df[c].notna().all()]
    df[col] = regression_impute(df, col, predictors)

# **Step 3: Multiple Imputation for Remaining Continuous Variables**
imputer = IterativeImputer(max_iter=10, random_state=42)
df[continuous_cols] = imputer.fit_transform(df[continuous_cols])

# Convert categorical variables back to integer format
df[categorical_cols] = df[categorical_cols].astype(int)

# Save final cleaned dataset
output_file = "/content/meps19_22_hybrid_clean.csv"
df.to_csv(output_file, index=False)

print(f"\nFinal hybrid-cleaned dataset saved as: {output_file}")

# Display basic dataset information
print("\nDataset Shape:", df.shape)  # Number of rows & columns
print("\nFirst 5 Rows:\n", df.head())  # Preview first few rows

# Check if there are any remaining missing values in the income variables
print("\nMissing values after imputation:")
print(df[["FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4"]].isna().sum())

# Check if any income values are still exactly 0 (since 0 could mean missing or legitimate response)
zero_income_count = (df[["FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4"]] == 0).sum().sum()
print(f"\nCases where income is exactly 0: {zero_income_count}")

# If needed, replace remaining NaNs with the median income value for each year
for col in ["FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4"]:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)

# Re-check to confirm no missing income
print("\nFinal check for missing income values:")
print(df[["FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4"]].isna().sum())

# Check if people with 0 income are students, dependents, or elderly
df[df["FAMINCY1"] == 0].head(20)

#Zero's appear to be valid.

# Display dataset summary again
print("\nDataset Shape:", df.shape)  # Number of rows & columns
print("\nFirst 5 Rows:\n", df.head())




NameError: name 'df' is not defined

In [ ]:
 # Prepare data for SQL and Tableau
# Data Transformation (Wide → Long Format)

import pandas as pd

# Display the first few rows of dataset in wide format (Before Transformation)
print("🔹 Wide Format (Before Transformation):")
display(df.head())

# Check summary statistics before transformation
print("\n🔹 Summary Statistics (Before Transformation):")
print(df.describe())

# Convert from wide to long format for expenditures, income, and insurance variables
df_long = df.melt(id_vars=["DUPERSID", "LONGWT"],
                  value_vars=[
                      "AGEY1X", "AGEY2X", "AGEY3X", "AGEY4X",
                      "FAMINCY1", "FAMINCY2", "FAMINCY3", "FAMINCY4",
                      "TOTEXPY1", "TOTEXPY2", "TOTEXPY3", "TOTEXPY4",
                      "TOTSLFY1", "TOTSLFY2", "TOTSLFY3", "TOTSLFY4",
                      "OBTOTVY1", "OBTOTVY2", "OBTOTVY3", "OBTOTVY4",
                      "ERTOTY1", "ERTOTY2", "ERTOTY3", "ERTOTY4",
                      "IPDISY1", "IPDISY2", "IPDISY3", "IPDISY4",
                      "RXEXPY1", "RXEXPY2", "RXEXPY3", "RXEXPY4",
                      "INSURCY1", "INSURCY2", "INSURCY3", "INSURCY4"
                  ],
                  var_name="Variable", value_name="Value")

# Extract Year from variable names (e.g., "TOTEXPY1" → 2019)
df_long["Year"] = df_long["Variable"].str.extract("(\d+)").astype(int) + 2018
df_long["Variable"] = df_long["Variable"].str.replace(r'Y\d+', '', regex=True)

# Pivot table to keep variables as separate columns
df_pivot = df_long.pivot(index=["DUPERSID", "Year", "LONGWT"], columns="Variable", values="Value").reset_index()

# Rename columns for clarity
df_pivot.rename(columns={
    "AGEX": "Age",
    "FAMINC": "Income",
    "TOTEXP": "Total_Expenditure",
    "TOTSLF": "Total_Self_Pay",
    "OBTOTV": "Outpatient_Visits",
    "ERTOT": "ER_Visits",
    "IPDIS": "Hospital_Stays",
    "RXEXP": "Prescription_Costs",
    "INSURC": "Insurance_Status"
}, inplace=True)

# Merge Re-coded Insurance Group
insurance_long = df.melt(id_vars=["DUPERSID"],
                          value_vars=["INSUR_GROUPY1", "INSUR_GROUPY2", "INSUR_GROUPY3", "INSUR_GROUPY4"],
                          var_name="Year", value_name="Insurance_Group")

# Extract Year from Insurance Group columns
insurance_long["Year"] = insurance_long["Year"].str.extract("(\d+)").astype(int) + 2018

# Merge Insurance Group into the long-format dataset
df_pivot = df_pivot.merge(insurance_long, on=["DUPERSID", "Year"], how="left")

# Display the first few rows of dataset in long format (After Transformation)
print("\n🔹 Long Format (After Transformation):")
display(df_pivot.head())

# Check summary statistics after transformation
print("\n🔹 Summary Statistics (After Transformation):")
print(df_pivot.describe())

# Ensure transformation was successful
print("\n🔹 Unique Years in Transformed Data:", df_pivot["Year"].unique())
print("\n🔹 Number of Rows After Transformation:", df_pivot.shape[0])

# Check if values are still intact after transformation
print("\n🔹 Checking Total Expenditure & Income Before and After Transformation:")
print("Before Transformation (Wide Format):")
print(df[["TOTEXPY1", "TOTEXPY2", "TOTEXPY3", "TOTEXPY4"]].describe())  # Summary before transformation
print("\nAfter Transformation (Long Format):")
print(df_pivot[["Total_Expenditure", "Income"]].describe())  # Summary after transformation

# Save the transformed dataset
output_file = "/content/meps19_22_long.csv"
df_pivot.to_csv(output_file, index=False)

print(f"\n📂 Transformed long-format dataset saved as: {output_file}")




🔹 Wide Format (Before Transformation):


,DUPERSID,AGEY1X,AGEY2X,AGEY3X,AGEY4X,FAMINCY1,FAMINCY2,FAMINCY3,FAMINCY4,INSURCY1,...,INSUR_GROUPY3,INSUR_GROUPY4,INCOME_GROUPY1,INCOME_GROUPY2,INCOME_GROUPY3,INCOME_GROUPY4,AGE_GROUPY1,AGE_GROUPY2,AGE_GROUPY3,AGE_GROUPY4
0,2460002101,74.0,75.0,76.0,77.0,13200.0,15500.0,10200.0,22000.0,6,...,Public,Public,Low (<25K),Low (<25K),Low (<25K),Low (<25K),65+,65+,65+,65+
1,2460006101,61.0,62.0,63.0,64.0,46196.0,44080.0,25788.0,5000.0,2,...,Public,Public,Lower-Middle (25K-50K),Lower-Middle (25K-50K),Lower-Middle (25K-50K),Low (<25K),50-64,50-64,50-64,50-64
2,2460006102,64.0,65.0,66.0,67.0,46196.0,44080.0,25788.0,30740.0,1,...,Public,Public,Lower-Middle (25K-50K),Lower-Middle (25K-50K),Lower-Middle (25K-50K),Lower-Middle (25K-50K),50-64,50-64,65+,65+
3,2460010101,26.0,27.0,28.0,29.0,25550.0,136334.0,101533.0,106483.0,1,...,Private,Private,Lower-Middle (25K-50K),High (>100K),High (>100K),High (>100K),18-34,18-34,18-34,18-34
5,2460018101,48.0,49.0,50.0,51.0,25483.0,32060.0,36000.0,45000.0,1,...,Private,Private,Lower-Middle (25K-50K),Lower-Middle (25K-50K),Lower-Middle (25K-50K),Lower-Middle (25K-50K),35-49,35-49,35-49,50-64



🔹 Summary Statistics (Before Transformation):
           DUPERSID       AGEY1X       AGEY2X       AGEY3X       AGEY4X  \
count  1.690000e+03  1690.000000  1690.000000  1690.000000  1690.000000   
mean   2.461705e+09    43.697050    44.676331    45.659172    46.635412   
std    9.614923e+05    22.773831    22.747383    22.717297    22.677457   
min    2.460002e+09     0.000000     1.000000     2.000000     3.000000   
25%    2.460896e+09    25.000000    26.000000    27.000000    28.000000   
50%    2.461696e+09    46.000000    47.000000    48.000000    49.000000   
75%    2.462551e+09    62.000000    63.000000    64.000000    65.000000   
max    2.463388e+09    85.000000    85.000000    85.000000    85.002556   

            FAMINCY1       FAMINCY2       FAMINCY3       FAMINCY4  \
count    1690.000000    1690.000000    1690.000000    1690.000000   
mean    73589.917160   75136.301183   76004.086391   79893.184615   
std     73509.197344   77957.428588   79979.637318   82755.250552   
m

,DUPERSID,Year,LONGWT,Age,ER_Visits,Income,Insurance_Status,Hospital_Stays,Outpatient_Visits,Prescription_Costs,Total_Expenditure,Total_Self_Pay,Insurance_Group
0,2460002101,2019,25259.75789,74.0,0.0,13200.0,6.0,0.0,12.0,2394.0,19573.0,62.0,Public
1,2460002101,2020,25259.75789,75.0,0.0,15500.0,6.0,0.0,8.0,6503.0,15474.0,101.0,Public
2,2460002101,2021,25259.75789,76.0,0.0,10200.0,6.0,0.0,1.0,5590.0,14290.0,274.0,Public
3,2460002101,2022,25259.75789,77.0,0.0,22000.0,6.0,0.0,2.0,1602.0,15766.0,109.0,Public
4,2460006101,2019,78297.80420,61.0,0.0,46196.0,2.0,0.0,24.0,4235.0,26827.0,4494.0,Public



🔹 Summary Statistics (After Transformation):
           DUPERSID         Year         LONGWT          Age    ER_Visits  \
count  6.760000e+03  6760.000000    6760.000000  6760.000000  6760.000000   
mean   2.461705e+09  2020.500000   63791.982875    45.166991     0.207396   
std    9.612789e+05     1.118117   61180.555703    22.750368     0.634747   
min    2.460002e+09  2019.000000    2101.940758     0.000000     0.000000   
25%    2.460895e+09  2019.750000   25725.932880    27.000000     0.000000   
50%    2.461696e+09  2020.500000   45252.598765    47.000000     0.000000   
75%    2.462551e+09  2021.250000   80450.812170    64.000000     0.000000   
max    2.463388e+09  2022.000000  472167.833500    85.002556    21.000000   

              Income  Insurance_Status  Hospital_Stays  Outpatient_Visits  \
count    6760.000000       6760.000000     6760.000000        6760.000000   
mean    76155.872337          2.304734        0.086538           7.016420   
std     78639.651696         

In [ ]:
import numpy as np
import scipy.stats as stats
import itertools

#recreate income bins
income_bins = [0, 25000, 50000, 75000, 100000, float("inf")]
income_labels = ["Low (<25K)", "Lower-Middle (25K-50K)", "Middle (50K-75K)", "Upper-Middle (75K-100K)", "High (>100K)"]

df_pivot_income["Income_Group"] = pd.cut(df_pivot_income["Income"], bins=income_bins, labels=income_labels)

print(df_pivot_income["Income_Group"].value_counts())  # Verify that groups exist


# ✅ Function: Z-test for Two Means
def z_test_means(sample1, sample2):
    """
    Performs a two-sample Z-test for means.
    Inputs:
      - sample1: First sample (e.g., 2019 healthcare expenditures)
      - sample2: Second sample (e.g., 2020 healthcare expenditures)
    Returns:
      - Z-score and p-value
    """

    # Remove NaN values
    sample1 = sample1.dropna()
    sample2 = sample2.dropna()

    # Compute means and standard deviations
    mean1, mean2 = np.mean(sample1), np.mean(sample2)
    std1, std2 = np.std(sample1, ddof=1), np.std(sample2, ddof=1)
    n1, n2 = len(sample1), len(sample2)

    # Check for zero division error (avoid empty groups)
    if n1 == 0 or n2 == 0:
        print("One of the groups has no data. Skipping comparison.\n")
        return None, None

    # Compute standard error of the difference
    se_diff = np.sqrt((std1**2 / n1) + (std2**2 / n2))

    # Compute Z-score
    z_score = (mean1 - mean2) / se_diff

    # Compute p-value (two-tailed test)
    p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

    # Print results
    print(f"Comparing: Mean1 = {mean1:.2f}, Mean2 = {mean2:.2f}")
    print(f"Z-score: {z_score:.4f}, p-value: {p_value:.4f}")

    if p_value < 0.05:
        print("Significant difference detected!\n")
    else:
        print("No significant difference.\n")

    return z_score, p_value

# Year-to-Year Expenditure Comparisons
print("\ Z-Tests for Yearly Differences in Total Expenditures:\n")
years = [2019, 2020, 2021, 2022]

for i in range(len(years) - 1):
    year1, year2 = years[i], years[i + 1]
    sample1 = df_pivot_income[df_pivot_income["Year"] == year1]["Total_Expenditure"]
    sample2 = df_pivot_income[df_pivot_income["Year"] == year2]["Total_Expenditure"]
    print(f"Z-test for {year1} vs. {year2}:")
    z_test_means(sample1, sample2)

# Income Group Comparisons
print("\ Z-Tests for Income Group Differences in Total Expenditures:\n")

income_groups = df_pivot_income["Income_Group"].dropna().unique()

for g1, g2 in itertools.combinations(income_groups, 2):
    sample1 = df_pivot_income[df_pivot_income["Income_Group"] == g1]["Total_Expenditure"]
    sample2 = df_pivot_income[df_pivot_income["Income_Group"] == g2]["Total_Expenditure"]

    print(f"Z-test for {g1} vs. {g2}:")
    z_test_means(sample1, sample2)

# Insurance Group Comparisons
print("\ Z-Tests for Insurance Group Differences in Total Expenditures:\n")

insurance_groups = df_pivot_income["Insurance_Group"].dropna().unique()

for g1, g2 in itertools.combinations(insurance_groups, 2):
    sample1 = df_pivot_income[df_pivot_income["Insurance_Group"] == g1]["Total_Expenditure"]
    sample2 = df_pivot_income[df_pivot_income["Insurance_Group"] == g2]["Total_Expenditure"]

    print(f"Z-test for {g1} vs. {g2}:")
    z_test_means(sample1, sample2)

#Results from data analyses
#1. Yearly Differences in Total Expenditures
#No significant changes between 2019-2021.
#Significant increase from 2021 to 2022 (p = 0.0030).


#2. Income Group Differences in Total Expenditures
#Significant differences exist between most income groups except:
#Lower-Middle (25K-50K) vs. Middle (50K-75K) (p = 0.73 → not significant)
#Lower-Middle (25K-50K) vs. High (>100K) (p = 0.09 → marginally significant)
#Middle (50K-75K) vs. Upper-Middle (75K-100K) (p = 0.59 → not significant)
#No significant difference between Low-Income (<25K) and High (>100K) (p = 0.93).

#Significant Differences Detected
#Low (<25K) vs. Lower-Middle (25K-50K) → Significant (p < 0.0001)
#Low (<25K) vs. Middle (50K-75K) → Significant (p = 0.0090)
#Low (<25K) vs. Upper-Middle (75K-100K) → Significant (p = 0.0013)
#High (>100K) vs. Upper-Middle (75K-100K) → Significant (p = 0.0401)

#3. Insurance Group Differences in Total Expenditures
#Public vs. Private: Large significant difference (p < 0.0001 → strong effect).
#Public vs. Uninsured: Significant difference (p = 0.0047 → moderate effect).
#Private vs. Uninsured: Large significant difference (p < 0.0001 → strong effect).




Income_Group
Low (<25K)                 15880
Lower-Middle (25K-50K)       906
Middle (50K-75K)             274
Upper-Middle (75K-100K)      143
High (>100K)                 139
Name: count, dtype: int64
\ Z-Tests for Yearly Differences in Total Expenditures:

Z-test for 2019 vs. 2020:
Comparing: Mean1 = 70922.99, Mean2 = 72472.79
Z-score: -1.1021, p-value: 0.2704
No significant difference.

Z-test for 2020 vs. 2021:
Comparing: Mean1 = 72472.79, Mean2 = 74354.69
Z-score: -1.2878, p-value: 0.1978
No significant difference.

Z-test for 2021 vs. 2022:
Comparing: Mean1 = 74354.69, Mean2 = 78985.40
Z-score: -2.9727, p-value: 0.0030
Significant difference detected!

\ Z-Tests for Income Group Differences in Total Expenditures:

Z-test for Low (<25K) vs. Lower-Middle (25K-50K):
Comparing: Mean1 = 77514.38, Mean2 = 65963.97
Z-score: 4.6524, p-value: 0.0000
Significant difference detected!

Z-test for Low (<25K) vs. Middle (50K-75K):
Comparing: Mean1 = 77514.38, Mean2 = 64017.01
Z-score: 2.6116